<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Read-data-from-tsv-source" data-toc-modified-id="Read-data-from-tsv-source-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Read data from tsv source</a></span></li><li><span><a href="#Connect-to-database" data-toc-modified-id="Connect-to-database-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Connect to database</a></span></li><li><span><a href="#Save-subreddit-category-info" data-toc-modified-id="Save-subreddit-category-info-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Save subreddit category info</a></span></li><li><span><a href="#Cleaning-function" data-toc-modified-id="Cleaning-function-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Cleaning function</a></span></li><li><span><a href="#Create-new-column-in-dataframe" data-toc-modified-id="Create-new-column-in-dataframe-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Create new column in dataframe</a></span></li><li><span><a href="#Load-spaCy" data-toc-modified-id="Load-spaCy-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Load spaCy</a></span></li><li><span><a href="#Iterate-over-all-rows-and-perform-NLP" data-toc-modified-id="Iterate-over-all-rows-and-perform-NLP-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Iterate over all rows and perform NLP</a></span></li><li><span><a href="#Check-results" data-toc-modified-id="Check-results-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Check results</a></span></li><li><span><a href="#Save-to-database" data-toc-modified-id="Save-to-database-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Save to database</a></span></li></ul></div>

# Preparations

In [1]:
import pandas as pd

## Read data from tsv source

In [2]:
df = pd.read_csv("covid19_tweets.csv")

Check if data is correct

In [3]:
df

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
0,ᏉᎥ☻լꂅϮ,astroworld,wednesday addams as a disney princess keepin i...,2017-05-26 05:46:42,624,950,18775,False,2020-07-25 12:27:21,If I smelled the scent of hand sanitizers toda...,NaN,Twitter for iPhone,False
1,Tom Basile 🇺🇸,"New York, NY","Husband, Father, Columnist & Commentator. Auth...",2009-04-16 20:06:23,2253,1677,24,True,2020-07-25 12:27:17,Hey @Yankees @YankeesPR and @MLB - wouldn't it...,NaN,Twitter for Android,False
2,Time4fisticuffs,"Pewee Valley, KY",#Christian #Catholic #Conservative #Reagan #Re...,2009-02-28 18:57:41,9275,9525,7254,False,2020-07-25 12:27:14,@diane3443 @wdunlap @realDonaldTrump Trump nev...,['COVID19'],Twitter for Android,False
3,ethel mertz,Stuck in the Middle,#Browns #Indians #ClevelandProud #[]_[] #Cavs ...,2019-03-07 01:45:06,197,987,1488,False,2020-07-25 12:27:10,@brookbanktv The one gift #COVID19 has give me...,['COVID19'],Twitter for iPhone,False
4,DIPR-J&K,Jammu and Kashmir,🖊️Official Twitter handle of Department of Inf...,2017-02-12 06:45:15,101009,168,101,False,2020-07-25 12:27:08,25 July : Media Bulletin on Novel #CoronaVirus...,"['CoronaVirusUpdates', 'COVID19']",Twitter for Android,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
74431,Laura Wolfrom,"Lexington, KY",The only things I collect are memories.,2010-09-24 02:01:15,85,586,1902,False,2020-08-04 03:13:29,So far this summer I have filled up my lawn mo...,"['COVID19', 'QuarantineLife']",Twitter for iPhone,False
74432,Professor Tonya M. Evans,😷 #stayathome,Law Prof @DickinsonLaw & Entrepreneur | Crypto...,2013-05-14 20:15:24,4289,1066,53569,False,2020-08-04 03:13:26,"ICYMI: REPLAY: #TechIntersect🧩 #16: Isaiah ""@B...","['TechIntersect', 'Bitcoin', 'COVID19']",Twitter Web App,False
74433,People's Daily app,"北京, 中华人民共和国",Our mission is to provide news and perspective...,2018-02-04 12:36:42,1413,102,16,False,2020-08-04 03:13:22,Community workers in Tianshan District of Urum...,"['China', 'Xinjiang']",Twitter Web App,False
74434,M0ser,NaN,Reagan conservative and attorney raised in the...,2014-02-18 03:46:28,2554,1733,129104,False,2020-08-04 03:13:19,If only we had a responsible media to warn us ...,['COVID19'],Twitter for iPhone,False


## Connect to database

In [4]:
import sqlite3
con = sqlite3.connect('selfposts.db')

## Save subreddit category info

In [5]:
pd.read_csv("covid19_tweets.csv").to_sql("categories", con)

ValueError: Table 'categories' already exists.

# Cleaning process

## Cleaning function

In [ ]:
import re
def clean(s):
    s = s.replace(r'<lb>', "\n")
    s = s.replace(r'<tab>', "\i")
    s = re.sub(r'<br */*>', "\n", s)
    s = s.replace("&lt;", "<").replace("&gt;", ">").replace("&amp;", "&")
    s = s.replace("&amp;", "&")
    # markdown urls
    s = re.sub(r'\(https*://[^\)]*\)', "", s)
    # normal urls
    s = re.sub(r'https*://[^\s]*', "", s)
    s = re.sub(r'_+', ' ', s)
    s = re.sub(r'"+', '"', s)
    return str(s)

## Create new column in dataframe

In [ ]:
df["selftext_clean"] = ''

# Iterate and clean

In [ ]:
for i, row in df.iterrows():
    df.at[i, "selftext_clean"] = clean(row.text)

Check results

In [ ]:
df.head()

# NLP

## Load spaCy

In [ ]:
import spacy
nlp = spacy.load('en')

## Iterate over all rows and perform NLP

In [ ]:
for i, row in df.iterrows():
    if i % 1000 == 0:
        print(i)
    if(row["selftext_clean"] and len(str(row["selftext_clean"])) < 1000000):
        doc = nlp(str(row["selftext_clean"]))
        adjectives = []
        nouns = []
        verbs = []
        lemmas = []

        for token in doc:
            lemmas.append(token.lemma_)
            if token.pos_ == "ADJ":
                adjectives.append(token.lemma_)
            if token.pos_ == "NOUN" or token.pos_ == "PROPN":
                nouns.append(token.lemma_)
            if token.pos_ == "VERB":
                verbs.append(token.lemma_)
                
        df.at[i, "selftext_lemma"] = " ".join(lemmas)                
        df.at[i, "selftext_nouns"] = " ".join(nouns)
        df.at[i, "selftext_adjectives"] = " ".join(adjectives)
        df.at[i, "selftext_verbs"] = " ".join(verbs)
        df.at[i, "selftext_nav"] = " ".join(nouns+adjectives+verbs)
        df.at[i, "no_tokens"] = len(lemmas)

## Check results

In [ ]:
df.head()

## Save to database

In [ ]:
df.to_sql('posts_nlp', con)